## Dependencies

In [1]:
from numpy import array, diag, diagflat, dot, tril, triu, linalg
import numpy as np


## User Input
The user_input() function collects the necessary information about the linear system  𝐴𝑥⃗=𝑏⃗ so that further calculations can be performed. The Jacobi method only works on square matrices (# rows = # columns), so the amount of rows and elements per row in  𝐴  must be equivalent. Vectors  𝑏⃗  and  𝑥⃗ , being single columns, will have as many elements as matrix  𝐴 has rows. For example, if  𝐴  has 3 rows, it will have 3 elements per row, and  𝑏⃗  and  𝑥⃗  will also be comprised of 3 elements. The function takes input specifically for the initial approximation of  𝑥⃗ :  𝑥⃗(0) . An initial guess of all zeros is recommended because it is simple, convenient, and consistent

In [2]:
def user_input():
    """
    For a linear system Ax=b, this function collects user input for
    square coefficient matrix A, column vector b, and initial guess x0.
    Returns augmented matrix A_with_b and initial guess.
    """

    global A, b, A_with_b, x0

    # matrix A input
    A_rows = int(input("Enter the number of rows in square matrix A: "))
    A = np.array([input(f"Enter row {i + 1}, separating elements by spaces: ").split() for i in range(A_rows)], dtype=float)
    print("\nMatrix A:")
    print(A)
    print()

    # vector b input
    b = np.array(input("Enter elements for vector b, separated by spaces: ").split(), dtype=float)
    b = b.reshape(-1, 1)
    print("\nVector b:")
    print(b)
    print()

    # Join b to the right of A
    A_with_b = np.hstack((A, b.reshape(-1, 1)))

    # vector x0 input
    x0 = np.array(input("Enter the initial guess for x, separated by spaces (suggested: all zeros): ").split(), dtype=float)
    x0 = x0.reshape(-1, 1)

    np.set_printoptions(formatter={'float': lambda x: f'{x:.0f}'})

    print("\nAugmented matrix A with b:")
    print(A_with_b)
    print("\nInitial guess x0:")
    print(x0)
    print()

    return A, x0


## Jacobi

### Spectral Radius (Jacobi)
The spectral radius $r_j$ of the matrix $T_j$, derived from components of user-given matrix $A$, determines whether or not $\vec{x}$ can be calculated using the Jacobi iterative method.$$T_j = D^{-1}(L + U)$$ The diagonal matrix $D$ must be invertible in order to calculate $T_j$ and $r_j$, so matrix $A$ must contain only non-zero entries along the diagonal. The spectral radius is the maximum of the absolute values of $T_j$'s eigenvalues. In each iteration of the Jacobi method, each approximation of $\vec{x}$ is proportional to the previous approximation, and the approximations will converge to the true solution. This means that the constant of proportionality (spectral radius, in this case) must be less than 1, or else each iteration will gradually cause $\vec{x}$ to approach infinity.

In [3]:
def spectral_radius_jacobi(A):
    """
    This function first determines if the spectral radius can be
    calculated by checking for zero entries in the diagonal of A.
    If any zeros are found, the code will stop running.
    Components of A (D, L, U) are extracted to find T_j and then calculate
    the spectral radius, which is assigned to variable r_j.
    If the spectral radius is greater than or equal to 1, the code will
    stop running.
    Returns spectral radius r_j.
    """

    np.set_printoptions(formatter={'float': lambda x: f'{x:.4f}'})
    global D, L, U, D_inv, T_j

    if not np.all(np.diag(A)) != 0: # make sure diagonals are nonzero before continuing
        print("The diagonal of matrix A includes zero entries, spectral radius undefined.")
        return False

    D = np.diag(diag(A)) # extract diagonal values
    L = -np.tril(A, k=-1) # extract values below diagonal
    U = -np.triu(A, k=1) # extract values above diagonal
    D_inv = np.linalg.inv(D) # inverse of diagonal matrix

    T_j = D_inv@(L + U)

    eigenvalues = np.linalg.eigvals(T_j) # eigenvalues of T_j
    abs_eigenvalues = np.abs(eigenvalues) # absolute values of eigenvalues
    max_eigenvalue = np.max(abs_eigenvalues) # maximum of the absolute values
    r_j = max_eigenvalue

    if r_j >= 1:
        print("x will not converge") # spectral radius must be less than 1
        return False
    else:
        print("x will converge via Jacobi method")
        print(f"Jacobi spectral radius: {r_j:.4f}")
        print()
    return r_j


### Number of Iterations (Jacobi)
The iterative process will stop when some convergence criterion is reached. We want the error between the true $\vec{x}$ and the estimated $\vec{x}$ to be very small, so the user is asked to define this threshold. Ideally, it should be 0.001 or smaller. Once this input is received, the estimated number of iterations $k$ can be calculated via the following formula: $$k > \frac{\log\left(\frac{{\text{threshold} \cdot (1-r_j)}}{{\|x^{(1)} - x^{(0)}\|}}\right)}{\log(r_j)}
$$This is just an estimate, but it does get very close to the true $\vec{x}$.

In [4]:
def num_iterations_jacobi(A):
    """
    This function first takes user input for the convergence threshold.
    The first iteration x1 and the difference x1-x0 are calculated to be
    used in the formula to find the estimated number of iterations k.
    Returns number of iterations k.
    """

    global threshold, c_j, x1
    threshold = float(input("Enter the convergence threshold (suggested: 0.001 or smaller): "))
    print()

    # calculate c
    c_j = D_inv@b

    # first iteration x1
    x1 = T_j@x0 + c_j

    diff = np.linalg.norm(x1-x0)

    k = int(np.ceil(np.log(threshold * (1 - r_j) / diff) / np.log(r_j)))
    print(f"Estimated number of iterations: {k}")
    print()
    return k


### Jacobi Iterative Method

The matrix form of the iterative algorithm follows this basic structure: $$x^{(k)} = Tx^{(k-1)} + c
$$ This function completes $k$ iterations as estimated in the previous function and then checks if the residual falls within the given error threshold. The residual is defined as $r = b - Ax
$. When the difference between the residuals of two successive iteration falls below the threshold, iterations are complete and the system has converged to a solution.

In [5]:
def jacobi_method(A, x0, threshold, k):

    x = x0.copy()  # Ensure x is a column vector
    total_iter = 0 # initialize total iterations counter variable
    prev_residual_norm = np.inf  # initialize with infinity to ensure the loop starts


    while True:
        total_iter += 1  # update counter by 1 each iteration
        x = T_j @ x + c_j  # update x using the Jacobi iteration formula
        residual = b - A @ x  # calculate current residual
        residual_norm = np.linalg.norm(residual)  # calculate the norm of the current residual
        prev_residual_norm = residual_norm
        residual_error = np.linalg.norm(residual - prev_residual_norm)


        if total_iter == k:
        # show user approximated x calculation after k iterations
            print(f"Solution after {total_iter} iterations:\n {x}\n")
            print(f"Residual: {residual_norm:.5f}")
            if residual_error <= threshold:
                print(f"Residual error is below or equal to the threshold after {total_iter} iterations.")
                print(f"Approximate solution:\n {x}\n")
                print()
                break

        if residual_error < threshold:
         # show user total iterations needed for residual difference to fall below threshold
        # and final x approximation
            print(f"Total number of iterations needed: {total_iter}\n")
            print(f"Approximate solution after {total_iter} iterations:\n {x}\n")
            print(f"Residual: {residual_norm:.5f}")
            print()
            break


    print(f"This error falls below the threshold of {threshold}.")
    print("x has been successfully calculated using the Jacobi iterative method.")

    return x


## Gauss-Seidel

### Spectral Radius (Gauss-Seidel)
The spectral radius $r_g$ of the matrix $T_g$, derived from components of user-given matrix $A$, determines whether or not $\vec{x}$ can be calculated using the Gauss iterative method.$$T_g = (D-L)^{-1}(U)$$ 

In [6]:
def spectral_radius_gauss(A):
    """
    Arugment: matrix A
    This function first determines if the spectral radius can be
    calculated by checking for zero entries in the diagonal of A.
    If any zeros are found, the code will stop running.
    Components of A (D, L, U) are extracted to find T_g and then calculate
    the spectral radius, which is assigned to variable r_g.
    If the spectral radius is greater than or equal to 1, the code will
    stop running.
    Returns spectral radius r_g.
    """

    np.set_printoptions(formatter={'float': lambda x: f'{x:.4f}'})
    global D, L, U, D_L_inv, T_g

    if not np.all(np.diag(A)) != 0: # make sure diagonals are nonzero before continuing
        print("The diagonal of matrix A includes zero entries, spectral radius undefined.")
        return False

    D = np.diag(diag(A)) # extract diagonal values
    L = -np.tril(A, k=-1) # extract values below diagonal
    U = -np.triu(A, k=1) # extract values above diagonal
    D_L_inv = np.linalg.inv(D-L) # inverse of diagonal matrix

    T_g = D_L_inv@U # calculate matrix T_g

    eigenvalues = np.linalg.eigvals(T_g) # eigenvalues of T_g
    abs_eigenvalues = np.abs(eigenvalues) # absolute values of eigenvalues
    max_eigenvalue = np.max(abs_eigenvalues) # maximum of the absolute values
    r_g = max_eigenvalue

    if r_g >= 1:
        print("x will not converge") # spectral radius must be less than 1
        return False
    else:
        print("x will converge via Gauss-Seidel method")
        print(f"Gauss-Seidel spectral radius: {r_g:.4f}")
        print()
    return r_g # r_g = spectral radius


### Number of Iterations (Gauss-Seidel)

In [7]:
def num_iterations_gauss(A):
    """
    Argument: matrix A
    This function first takes user input for the convergence threshold.
    The first iteration x1 and the difference x1-x0 are calculated to be
    used in the formula to find the estimated number of iterations k.
    Returns number of iterations k.
    """

    global threshold, c_g, x1

    # threshold input
    threshold = float(input("Enter the convergence threshold (suggested: 0.001 or smaller): "))
    print()

    c_g = D_L_inv@b # calculate c_g

    x1 = T_g@x0 + c_g # first iteration x1

    diff = np.linalg.norm(x1-x0) # difference between x1 and x0

    k = int(np.ceil(np.log(threshold * (1 - r_g) / diff) / np.log(r_g))) # calculate k
    print(f"Estimated number of iterations: {k}")
    print()
    return k


### Gauss-Seidel Iterative Method

In [8]:
def gauss_method(A, x0, threshold, k):
    """
    Arguments: matrix A, initial guess x0, threshold
    This function first calculates the true x solution in order to
    determine the error between x_true and approximated x.
    The iterations take place inside a while loop that keep track of
    the total number of iterations.
    When k iterations are calculated, the approximate solution is displayed.
    Iterations continue until the error falls below the threshold.
    Returns final x approximation.
    """

    x = x0.copy()  # Ensure x is a column vector
    total_iter = 0 # initialize total iterations counter variable
    prev_residual_norm = np.inf  # initialize with infinity to ensure the loop starts

    while True:
        total_iter += 1  # update counter by 1 each iteration
        x = T_g @ x + c_g  # update x using the Gauss-Seidel iteration formula
        residual = b - A @ x  # calculate current residual
        residual_norm = np.linalg.norm(residual)  # calculate the norm of the current residual
        prev_residual_norm = residual_norm
        residual_error = np.linalg.norm(residual - prev_residual_norm)


        if total_iter == k:
        # show user approximated x calculation after k iterations
            print(f"Solution after {total_iter} iterations:\n {x}\n")
            print(f"Residual: {residual_norm:.5f}")
            if residual_error <= threshold:
                print(f"Residual error is below or equal to the threshold after {total_iter} iterations.")
                print(f"Approximate solution:\n {x}\n")
                print()
                break

        if residual_error < threshold:
         # show user total iterations needed for residual difference to fall below threshold
        # and final x approximation
            print(f"Total number of iterations needed: {total_iter}\n")
            print(f"Approximate solution after {total_iter} iterations:\n {x}\n")
            print(f"Residual: {residual_norm:.5f}")
            print()
            break


    print(f"This error falls below the threshold of {threshold}.")
    print("x has been successfully calculated using the Gauss-Seidel iterative method.")

    return x


## Successive Over-Relaxation (SOR)

### Omega

The SOR method is very similar to Gauss-Seidel, except it has a weight parameter $\omega$ which is a constant that speeds up convergence. In order for the solution to converge, $\omega$ must be between 0 and 2. It is calculated using the Jacobi matrix's spectral radius: $$\omega = \frac{2}{1 + \sqrt{1 - (r_j)^2}}$$

In [9]:
def omega(A):
    w = 2 / (1 + np.sqrt(1-(r_j)**2))
    print(f"omega: {w:.4f}")
    if (w>0 and w<2):
        print("The omega value is valid, since it is between 0 and 2.")
        print()
    else:
        print("x will not converge, due to invalid omega.")

    return w


### Spectral Radius (SOR)
The spectral radius $r$ of the matrix $T_\omega$, derived from components of user-given matrix $A$, determines whether or not $\vec{x}$ can be calculated using the Gauss iterative method.$$T_\omega = (D - \omega L)^{-1} \left[ (1 - \omega)D + \omega U \right]$$


In [10]:
def spectral_radius_sor(A, w):
    global T_w

    np.set_printoptions(formatter={'float': lambda x: f'{x:.4f}'})
    if not np.all(np.diag(A)) != 0: # make sure diagonals are nonzero before continuing
        print("The diagonal of matrix A includes zero entries, spectral radius undefined.")
        return False

    D = np.diag(diag(A)) # extract diagonal values
    L = -np.tril(A, k=-1) # extract values below diagonal
    U = -np.triu(A, k=1) # extract values above diagonal

    T_w = np.linalg.inv(D - w * L) @ ((1 - w) * D + w * U) # calculate matrix T_w

    eigenvalues = np.linalg.eigvals(T_w) # eigenvalues of T_w
    abs_eigenvalues = np.abs(eigenvalues) # absolute values of eigenvalues
    max_eigenvalue = np.max(abs_eigenvalues) # maximum of the absolute values
    r_w = max_eigenvalue

    if r_w >= 1:
        print("x will not converge via SOR method") # spectral radius must be less than 1
        return False
    else:
        print("x will converge via SOR method")
        print(f"SOR spectral radius: {r_w:.4f}")
        print()

    return r_w # r_w = spectral radius

### Number of Iterations (SOR)

In [11]:
def num_iterations_sor(A):
    """
    This function first takes user input for the convergence threshold.
    The first iteration x1 and the difference x1-x0 are calculated to be
    used in the formula to find the estimated number of iterations k.
    Returns number of iterations k.
    """

    global threshold, c_w, x1
    threshold = float(input("Enter the convergence threshold (suggested: 0.001 or smaller): "))
    print()

    # calculate c
    c_w = w * np.linalg.inv(D - w * L) @b

    # first iteration x1
    x1 = T_w@x0 + c_w

    diff = np.linalg.norm(x1-x0)

    k = int(np.ceil(np.log(threshold * (1 - r_w) / diff) / np.log(r_w)))
    print(f"Estimated number of iterations: {k}")
    print()
    return k


### SOR Iterative Method

In [12]:
def sor_method(A, x0, w, threshold, k):
    """
    Arguments: matrix A, initial guess x0, relaxation parameter omega,
               threshold, number of iterations k, true solution vector b
    This function iteratively calculates the solution of Ax=b using the Successive Over-Relaxation (SOR) method.
    """

    x = x0.copy()  # Ensure x is a column vector
    total_iter = 0  # initialize total iterations counter variable
    prev_residual_norm = np.inf  # initialize with infinity to ensure the loop starts

    while True:
        total_iter += 1  # update counter by 1 each iteration
        x = T_w @ x + c_w
        residual = b - A @ x  # calculate current residual
        residual_norm = np.linalg.norm(residual)  # calculate the norm of the current residual
        prev_residual_norm = residual_norm
        residual_error = np.linalg.norm(residual - prev_residual_norm)


        if total_iter == k:
        # show user approximated x calculation after k iterations
            print(f"Solution after {total_iter} iterations:\n {x}\n")
            print(f"Residual: {residual_norm:.5f}")
            if residual_error <= threshold:
                print(f"Residual error is below or equal to the threshold after {total_iter} iterations.")
                print(f"Approximate solution:\n {x}\n")
                print()
                break

        if residual_error < threshold:
        # show user total iterations needed for residual difference to fall below threshold
        # and final x approximation
            print(f"Total number of iterations needed: {total_iter}\n")
            print(f"Approximate solution after {total_iter} iterations:\n {x}\n")
            print(f"Residual: {residual_norm:.5f}")
            print()
            break


    print(f"This error falls below the threshold of {threshold}.")
    print("x has been successfully calculated using the SOR iterative method.")

    return x

### Test

In order to decide which method to use, we must first calculate the spectral radius for all three methods. After comparing all the spectral radii, the method with the smallest spectral radius will be chosen and its functions are run.

In [ ]:

while True:
    A, x0 = user_input()

    r_j = spectral_radius_jacobi(A)
    if not r_j:
        print("x cannot be calculated via the Jacobi method.")
        break

    r_g = spectral_radius_gauss(A)
    if not r_g:
        print("x cannot be calculated via the Gauss-Seidel method.")
        break

    w = omega(A)
    if not w:
        print("x cannot be calculated via the SOR method.")
        break
    r_w = spectral_radius_sor(A,w)
    if not r_w:
        print("x cannot be calculated via the SOR method.")
        break

    if (r_j<r_g and r_j<r_w):
        print("x will converge faster using the Jacobi method because its spectral radius is smaller.")
        print()
        k = num_iterations_jacobi(A)
        x = jacobi_method(A, x0, threshold, k)
        break
    if (r_g < r_j and r_g < r_w):
        print("x will converge faster using the Gauss-Seidel method because its spectral radius is smaller")
        print()
        k = num_iterations_gauss(A)
        x = gauss_method(A, x0, threshold, k)
        break
    if (r_w < r_j and r_w < r_g):
        print("x will converge faster using the SOR method because its spectral radius is smaller")
        print()
        k = num_iterations_sor(A)
        x = sor_method(A, x0, w, threshold, k)
        break

